<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd 
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# fix random seed for reproducibility
np.random.seed(42)

In [2]:
df = pd.read_csv('/Users/TomasFox/Downloads/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [13]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [4]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0).astype(int)

In [5]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.265086
std,0.368612,24.559481,30.090047,2266.790864
min,0.000000,0.000000,18.250000,0.000000
25%,0.000000,9.000000,35.500000,398.000000
50%,0.000000,29.000000,70.350000,1394.000000
75%,0.000000,55.000000,89.850000,3786.500000
max,1.000000,72.000000,118.750000,8684.000000


In [6]:
y = df['Churn']

In [7]:
X = df.drop(['Churn','customerID'], axis=1)

In [8]:
from category_encoders.ordinal import OrdinalEncoder


encode = OrdinalEncoder()
X = encode.fit_transform(X)

In [17]:
encode = OrdinalEncoder()
y = encode.fit_transform(y.values)

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

[[-1.00955867 -0.43991649 -1.03453023 ... -1.14519806 -1.16032292
  -0.9927801 ]
 [ 0.99053183 -0.43991649  0.96662231 ... -0.27474372 -0.25962894
  -0.17217856]
 [ 0.99053183 -0.43991649  0.96662231 ... -0.27474372 -0.36266036
  -0.95792659]
 ...
 [-1.00955867 -0.43991649 -1.03453023 ... -1.14519806 -1.1686319
  -0.85292489]
 [ 0.99053183  2.27315869 -1.03453023 ... -0.27474372  0.32033821
  -0.87057223]
 [ 0.99053183 -0.43991649  0.96662231 ...  0.59571063  1.35896134
   2.01388628]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [19]:
X.shape, y.shape

((7043, 19), (7043, 1))

In [23]:
y = y.values

In [35]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold


# define k-fold cross validation test harness
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
cvscores = []
for train, test in kfold.split(X, y):
    
  # create model
  def create_model():
        model = Sequential()
        model.add(Dense(19, input_dim=19, activation='relu'))
        model.add(Dense(19, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

  # compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model 
    
  # in order to hyperparameter tune we need to pass in KerasClassifier
  model_class = KerasClassifier(build_fn=create_model, verbose=0)
    

  # fit the model
  model.fit(X[train], y[train], epochs=20, batch_size=10, verbose=0)


  # evaluate the model
  scores = model.evaluate(X[test], y[test], verbose=0)
  print(f'{model.metrics_names[1]}: {(scores[1]*100):.2f}%') 
  cvscores.append(scores[1]*100)
print(f'{np.mean(cvscores):.2f}% +/- {np.std(cvscores):.2f}%')

acc: 73.45%
acc: 73.47%
73.46% +/- 0.01%


In [38]:
# define the grid search parameters
param_grid = {'batch_size': [20, 40],
              'epochs': [5, 10]}

# Create Grid Search
grid = GridSearchCV(estimator=model_class, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8015050217541005 using {'batch_size': 20, 'epochs': 5}
Means: 0.8015050217541005, Stdev: 0.0016315872158934438 with: {'batch_size': 20, 'epochs': 5}
Means: 0.8007951117739934, Stdev: 0.003539992546505134 with: {'batch_size': 20, 'epochs': 10}
Means: 0.8003691708129288, Stdev: 0.0003831012395808192 with: {'batch_size': 40, 'epochs': 5}
Means: 0.8002271865234445, Stdev: 0.0047180772840929855 with: {'batch_size': 40, 'epochs': 10}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?